In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import pandas as pd

from exp.exp_informer import Exp_Informer
import matplotlib.pyplot as plt

from utils.masking import TriangularCausalMask, ProbMask
from models.encoder import Encoder, EncoderLayer, ConvLayer, EncoderStack
from models.decoder import Decoder, DecoderLayer
from models.attn import FullAttention, ProbAttention, AttentionLayer
from models.embed import DataEmbedding, PositionalEmbedding, TokenEmbedding, FixedEmbedding, TemporalEmbedding, TimeFeatureEmbedding

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

from utils.tools import StandardScaler
from utils.timefeatures import time_features

from data.data_loader import Dataset_ETT_hour, Dataset_ETT_minute, Dataset_Custom, Dataset_Pred
from exp.exp_basic import Exp_Basic
from models.model import Informer, InformerStack

from utils.tools import EarlyStopping, adjust_learning_rate
from utils.metrics import metric

from scipy.special import softmax
import torch
import pandas as pd
import numpy as np
import math

from ray import tune, air
from ray.tune import ResultGrid

# Additional packages
# Create dataset
df_raw = pd.read_csv('data/processed/SRL/SRL_NEG_00_04.csv')

train =  Dataset_Custom('data/processed/SRL/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price')
test =  Dataset_Custom('data/processed/SRL/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price',flag='test')
vali =  Dataset_Custom('data/processed/SRL/', freq= 'd', data_path='SRL_NEG_00_04.csv', target='capacity_price', flag='val')

In [16]:
experiment_path = f"ray_tune/tune_SRL_POS_00_04_01-03-2023_08-28-06/"
print(f"Loading results from {experiment_path}...")

restored_tuner = tune.Tuner.restore(experiment_path)
result_grid = restored_tuner.get_results()

best = result_grid.get_best_result(metric='revenue', mode='max')
# best.config
best.metrics

Loading results from ray_tune/tune_SRL_POS_00_04_01-03-2023_08-28-06/...


{'revenue': 396.62,
 'loss': 0.02333561,
 'time_this_iter_s': 50.07645034790039,
 'should_checkpoint': True,
 'done': True,
 'timesteps_total': None,
 'episodes_total': None,
 'training_iteration': 1,
 'trial_id': '39ea3bb6',
 'experiment_id': '6d05c8329495477db2fecfadc749eb63',
 'date': '2023-03-01_08-37-08',
 'timestamp': 1677656228,
 'time_total_s': 50.07645034790039,
 'pid': 7744,
 'hostname': 'azwewpcdml-1',
 'node_ip': '127.0.0.1',
 'config': {'learning_rate': 0.0009247988557174399,
  'train_epochs': 9,
  'alpha': 1.2,
  'seq_len': 112,
  'label_seq_len_ratio': 0.5,
  'batch_size': 16},
 'time_since_restore': 50.07645034790039,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 1,
 'warmup_time': 0.0,
 'experiment_tag': '38_alpha=1.2000,batch_size=16,label_seq_len_ratio=0.5000,learning_rate=0.0009,seq_len=112,train_epochs=9'}

In [4]:
best

Result(metrics={'revenue': 579.33, 'loss': 0.13410503, 'should_checkpoint': True, 'done': True, 'trial_id': '9c96a32e', 'experiment_tag': '16_alpha=5.7000,learning_rate=0.0082,pred_seq_len_ratio=0.5000,seq_len=98,train_epochs=13'}, error=None, log_dir=WindowsPath('c:/codes/srl_informer/ray_tune/tune_SRL_NEG_00_04_28-02-2023_08-11-40/trainable_9c96a32e_16_alpha=5.7000,learning_rate=0.0082,pred_seq_len_ratio=0.5000,seq_len=98,train_epochs=13_2023-02-28_08-14-02'))

In [23]:
torch.load(best.checkpoint._local_path)

PermissionError: [Errno 13] Permission denied: 'c:\\codes\\srl_informer\\ray_tune\\tune_SRL_NEG_00_04_24-02-2023_06-56-42\\trainable_b81eb0f3_3_alpha=9.0000,learning_rate=0.0014,seq_len=56,train_epochs=6_2023-02-24_06-57-14\\checkpoint_000000'

'c:\\codes\\srl_informer\\ray_tune\\tune_SRL_NEG_00_04_24-02-2023_06-56-42\\trainable_b81eb0f3_3_alpha=9.0000,learning_rate=0.0014,seq_len=56,train_epochs=6_2023-02-24_06-57-14\\checkpoint_000000'